In [1]:
import os
import time
import random
import argparse
import numpy as np
from tqdm import tqdm
from mixup import mixup_data, mixup_criterion
from utilities import AverageTracker, get_optimizer
from pytorch_nn import Lipreading1
from datasetloadingpy import dataloaders

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def evaluate(model, dset_loader, criterion):

    model.eval()

    running_loss = 0.
    running_corrects = 0.

    with torch.no_grad():
        for batch_idx, (input, lengths, labels) in enumerate(tqdm(dset_loader)):
            logits = model(input.unsqueeze(1).cuda(), lengths=lengths)
            _, preds = torch.max(F.softmax(logits, dim=1).data, dim=1)
            running_corrects += preds.eq(labels.cuda().view_as(preds)).sum().item()

            loss = criterion(logits, labels.cuda())
            running_loss += loss.item() * input.size(0)

    print('{} in total\tCR: {}'.format( len(dset_loader.dataset), running_corrects/len(dset_loader.dataset)))
    return running_corrects/len(dset_loader.dataset), running_loss/len(dset_loader.dataset)

In [3]:
def train_loop(model, dataloader, criterion, epoch, optimizer):

    data_time = AverageTracker()
    batch_time = AverageTracker()

    mixup_alpha = 0.4

    print("Current Epoch: " + str(epoch))

    model.train()
    running_loss = 0.
    running_corrects = 0.
    running_all = 0.

    for batch_idx, (input, lengths, labels) in enumerate(dataloader):

        input, labels_a, labels_b, lam = mixup_data(input, labels, mixup_alpha)
        labels_a, labels_b = labels_a.cuda(), labels_b.cuda()

        optimizer.zero_grad()

        #logits = model(input.unsqueeze(1).cuda(), lengths=lengths)
        logits = model(input.unsqueeze(1).cuda())

        loss_func = mixup_criterion(labels_a, labels_b, lam)
        loss = loss_func(criterion, logits)

        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # -- compute running performance
        _, predicted = torch.max(F.softmax(logits, dim=1).data, dim=1)
        running_loss += loss.item()*input.size(0)
        running_corrects += lam * predicted.eq(labels_a.view_as(predicted)).sum().item() + (1 - lam) * predicted.eq(labels_b.view_as(predicted)).sum().item()
        running_all += input.size(0)
    print("Running Loss: {}, Running Corrects: {}, Running All: {}".format(running_loss,running_corrects,running_all))

    return model

In [4]:
data_path = '/home/taylorpap/Bootcamp/CroppedLRW'
temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT',
                   'WEATHER']
new_temp_words_list = ['BUDGET']
datasets = dataloaders(data_dir=data_path, label_fp=temp_words_list, batch_size=32)


Partition train loaded
Partition test loaded


In [5]:
train_features, train_lengths, train_labels = next(iter(datasets['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 29, 88, 88])
Labels batch shape: torch.Size([32])


In [6]:
test_model = Lipreading1(len(temp_words_list))

In [7]:
if torch.cuda.is_available():
    test_model.cuda()

In [11]:
test_model

Lipreading1(
  (frontend3D): Sequential(
    (0): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (max_pool1): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  (trunk): ResNet(
    (0): Sequential(
      (0): ResBlock(
        (convs): Sequential(
          (0): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (2): ConvLayer(
            (0)

In [8]:
optimizer = get_optimizer('sgd', optim_policies=test_model.parameters())
criterion = nn.CrossEntropyLoss()
#scheduler = CosineScheduler(args.lr, args.epochs)
epoch=0

In [9]:
while epoch < 5:
    model = train_loop(test_model, datasets['train'], criterion, epoch, optimizer)
    epoch +=1

Current Epoch: 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 878.00 MiB (GPU 0; 23.68 GiB total capacity; 20.26 GiB already allocated; 646.75 MiB free; 20.76 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF